## 1. Data Description

### Sessions dataframe:
- 1535 observations, represent individual player sessions

- 5 variables: hashed email, start time, end time, original start time, original end time

- Start times and end times are quantitative variables, represent date and time of day that players start and end their playing session

- Hashed email is qualitative (categorical), an encrypted version of player's email to keep their data anonymous

- Original start and end time are quantitative variables and are a standardized way of writing the time (UNIX/seconds since January 1 1970 at 0:00)

Note: we will not be using the players dataframe, as for the question we have chosen, only the sessions dataframe will contain the information needed to find the time windows of most traffic.
 
#### Issues:

- The data is not in tidy format (start and end times can be divided into separate columns such as date, time, year, etc.)

- The hashed email is not useful information for this intended question as it does not concisely identify the players.

- Original start and end time are not in a format that can be used for data analysis

- Start and end time are not in a datetime format which pandas requires to mannipulate time variables

- Dataframe has a lot of observations (1500+) which may not be entirely necessary for our purposes and might not be useful in visualizations, it would be helpful for the data to be sorted and grouped based on their time frames.

## 2. Question
Question 2: We are interested in demand forecasting, namely, what time windows are most likely to have large number of simultaneous players. This is because we need to ensure that the number of licenses on hand is sufficiently large to accommodate all parallel players with high probability.
- The sessions dataframe will be useful for addressing this question because this dataframe contains the time frame windows that players have played. The response variable of interest is the commonality of players playing/number of active players, and the explanatory variable is the time.
- To wrangle the data, we will likely have to tidy the data, separating it into day, month, and year. We will also need to convert the time into datetime format in order to manipulate it (turn into number of minutes, hours, etc.). We will also drop the original time columns and hashed email, as we are not going to use this in our final model.
- Possible specific questions to explore: What time of day is most popular/has the most traffic, how long do players typically play for, what length of playing time is the most common

## 3. Exploratory Data Analysis and Visualization (wrangling and tidying)

In [1]:
import altair as alt
import pandas as pd

In [2]:
sessions_data = pd.read_csv('sessions.csv')
sessions_data

,hashedEmail,start_time,end_time,original_start_time,original_end_time
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,30/06/2024 18:12,30/06/2024 18:24,1.720000e+12,1.720000e+12
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,17/06/2024 23:33,17/06/2024 23:46,1.720000e+12,1.720000e+12
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,25/07/2024 17:34,25/07/2024 17:57,1.720000e+12,1.720000e+12
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,25/07/2024 03:22,25/07/2024 03:58,1.720000e+12,1.720000e+12
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,25/05/2024 16:01,25/05/2024 16:12,1.720000e+12,1.720000e+12
...,...,...,...,...,...
1530,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,10/05/2024 23:01,10/05/2024 23:07,1.720000e+12,1.720000e+12
1531,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e...,01/07/2024 4:08,01/07/2024 4:19,1.720000e+12,1.720000e+12
1532,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,28/07/2024 15:36,28/07/2024 15:57,1.720000e+12,1.720000e+12
1533,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,25/07/2024 06:15,25/07/2024 06:22,1.720000e+12,1.720000e+12


In [3]:
sessions_data[['start_date','start_time_new']] = sessions_data['start_time'].str.split(' ',expand=True)
sessions_data[['end_date','end_time_new']] = sessions_data['end_time'].str.split(' ',expand=True)
sessions2 = sessions_data.drop(columns = ['start_time','end_time','original_start_time','original_end_time','hashedEmail'])
sessions2
#trying to split up time and date to visualize separately

,start_date,start_time_new,end_date,end_time_new
0,30/06/2024,18:12,30/06/2024,18:24
1,17/06/2024,23:33,17/06/2024,23:46
2,25/07/2024,17:34,25/07/2024,17:57
3,25/07/2024,03:22,25/07/2024,03:58
4,25/05/2024,16:01,25/05/2024,16:12
...,...,...,...,...
1530,10/05/2024,23:01,10/05/2024,23:07
1531,01/07/2024,4:08,01/07/2024,4:19
1532,28/07/2024,15:36,28/07/2024,15:57
1533,25/07/2024,06:15,25/07/2024,06:22


In [4]:
sessions2_small = sessions2.head(200)
#reduce the number of data on the plot to better see a trend instead of very clustered 

In [5]:
bar_plot = alt.Chart(sessions2_small).mark_bar().encode(
    x=alt.X('start_date:T').title('Start Date'),
    y=alt.Y('count()').title('Number of sessions started')
).properties(title='Distribution of sessions started throughout the year')
bar_plot

alt.Chart(...)

This plot is representing the date which each player starts their session on the x axis and the number of players who start their session on that date on the y axis. The bar graph shows a comparison of these amounts throughout 2024, but is not extremely effective because of how specific these days might be. It would be better to group these dates to give a more general trend. 

In [6]:
scatter_plot = alt.Chart(sessions2_small).mark_point().encode(
    x=alt.X('start_date:T').title('Start Date'),
    y=alt.Y('count()').title('Number of sessions started')
).properties(title='Number of sessions started over time')
scatter_plot

alt.Chart(...)

This plot is the same as the previous plot but in scatter format. This also shows how the data should be manipulated or grouped to better see a trend. This also shows that a bar plot to compare amounts between different times of the year/times of the day might be more effective. However, I think a scatter plot could be used to answer the question by showing a visualization of time throughout the day on the x axis and have scattered points across the visualization representing when each player logs into their session. This could visually represent what times are most clustered. The y axis could represent the number of players logged in at that specific time.

In [7]:
line_plot = alt.Chart(sessions2_small).mark_line().encode(
    x=alt.X('start_date:T').title('Start Date').scale(zero=False),
    y=alt.Y('count()').title('Number of sessions started').scale(domain=(0,5))
).properties(title='Number of sessions started over time')
line_plot

alt.Chart(...)

This is the same plot as the previous 2 but as a line. Again, this shows that when seeing what time is most popular for players to log in, a bar graph might be a better fit to visualize the amounts when the time is grouped into hours, months, etc. Overall, these visualizations tell me that I need to manipulate the data through grouping or converting time to make the trend more visible. The current visualizations do not say much about general trends, so it might be beneficial to extract specific hours and minutes rather than look at the entire date. The pandas documentation shows that this can be done by converting the start time and end time (which are currently strings) to datetime objects. This will allow python to extract specific values from the time.

In [8]:
sessions_data['start_time'] = pd.to_datetime(sessions_data['start_time'],format='%d/%m/%Y %H:%M')
sessions_data['end_time'] = pd.to_datetime(sessions_data['end_time'],format='%d/%m/%Y %H:%M')
sessions_data = sessions_data.drop(columns = ['hashedEmail','original_start_time','original_end_time','start_date','start_time_new','end_date','end_time_new'])

In [9]:
#create new columns to see how long players typically play for
sessions_data['total_minutes_start'] = sessions_data['start_time'].dt.hour * 60 + sessions_data['start_time'].dt.minute
sessions_data['total_minutes_end'] = sessions_data['end_time'].dt.hour * 60 + sessions_data['end_time'].dt.minute
sessions_data

,start_time,end_time,total_minutes_start,total_minutes_end
0,2024-06-30 18:12:00,2024-06-30 18:24:00,1092,1104.0
1,2024-06-17 23:33:00,2024-06-17 23:46:00,1413,1426.0
2,2024-07-25 17:34:00,2024-07-25 17:57:00,1054,1077.0
3,2024-07-25 03:22:00,2024-07-25 03:58:00,202,238.0
4,2024-05-25 16:01:00,2024-05-25 16:12:00,961,972.0
...,...,...,...,...
1530,2024-05-10 23:01:00,2024-05-10 23:07:00,1381,1387.0
1531,2024-07-01 04:08:00,2024-07-01 04:19:00,248,259.0
1532,2024-07-28 15:36:00,2024-07-28 15:57:00,936,957.0
1533,2024-07-25 06:15:00,2024-07-25 06:22:00,375,382.0


In [10]:
#subtract total minutes at the start from total minutes at the end to get time played
#use .abs() to get the absolute value of the difference to account for potential negative values
sessions_data['time_played_mins'] = (sessions_data['total_minutes_end'] - sessions_data['total_minutes_start']).abs()
sessions_data

,start_time,end_time,total_minutes_start,total_minutes_end,time_played_mins
0,2024-06-30 18:12:00,2024-06-30 18:24:00,1092,1104.0,12.0
1,2024-06-17 23:33:00,2024-06-17 23:46:00,1413,1426.0,13.0
2,2024-07-25 17:34:00,2024-07-25 17:57:00,1054,1077.0,23.0
3,2024-07-25 03:22:00,2024-07-25 03:58:00,202,238.0,36.0
4,2024-05-25 16:01:00,2024-05-25 16:12:00,961,972.0,11.0
...,...,...,...,...,...
1530,2024-05-10 23:01:00,2024-05-10 23:07:00,1381,1387.0,6.0
1531,2024-07-01 04:08:00,2024-07-01 04:19:00,248,259.0,11.0
1532,2024-07-28 15:36:00,2024-07-28 15:57:00,936,957.0,21.0
1533,2024-07-25 06:15:00,2024-07-25 06:22:00,375,382.0,7.0


In [11]:
time_played_plot = alt.Chart(sessions_data).mark_bar().encode(
    x=alt.X('time_played_mins').title('Time played (min)').bin(maxbins=20),
    y=alt.Y('count()').title('Number of players')
).properties(title='Distribution of player count based on time played')
time_played_plot 
# shows how long players typically play for

alt.Chart(...)

This visualization shows that the majority of players play for less than 100 minutes, showing a lesser likelihood of longer play times.

In [12]:
sessions_data['hour'] = pd.to_datetime(sessions_data['start_time'], format='%H:%M').dt.hour
sessions_data['day'] = pd.to_datetime(sessions_data['start_time'], format='%d/%m/%Y').dt.day
sessions_data['month'] = pd.to_datetime(sessions_data['start_time'], format='%d/%m/%Y').dt.month
sessions_data['year'] = pd.to_datetime(sessions_data['start_time'],format='%d/%m/%Y').dt.year
sessions_data
#split up start_time into hour, date, month, year that they started playing for that session

,start_time,end_time,total_minutes_start,total_minutes_end,time_played_mins,hour,day,month,year
0,2024-06-30 18:12:00,2024-06-30 18:24:00,1092,1104.0,12.0,18,30,6,2024
1,2024-06-17 23:33:00,2024-06-17 23:46:00,1413,1426.0,13.0,23,17,6,2024
2,2024-07-25 17:34:00,2024-07-25 17:57:00,1054,1077.0,23.0,17,25,7,2024
3,2024-07-25 03:22:00,2024-07-25 03:58:00,202,238.0,36.0,3,25,7,2024
4,2024-05-25 16:01:00,2024-05-25 16:12:00,961,972.0,11.0,16,25,5,2024
...,...,...,...,...,...,...,...,...,...
1530,2024-05-10 23:01:00,2024-05-10 23:07:00,1381,1387.0,6.0,23,10,5,2024
1531,2024-07-01 04:08:00,2024-07-01 04:19:00,248,259.0,11.0,4,1,7,2024
1532,2024-07-28 15:36:00,2024-07-28 15:57:00,936,957.0,21.0,15,28,7,2024
1533,2024-07-25 06:15:00,2024-07-25 06:22:00,375,382.0,7.0,6,25,7,2024


In [13]:
hours_plot = alt.Chart(sessions_data).mark_bar().encode(
    x=alt.X('hour').title('Hour of the day').bin(maxbins=20),
    y=alt.Y('count()').title('Number of players')
).properties(title='Distribution of player count throughout the day')
hours_plot

alt.Chart(...)

This visualization shows the distribution of amount of players that choose to log in throughout the day. It shows that the time that the most players log in is at 2am-4am, and it is least likely to find players logging in at 10am-2pm. There is not an overwhelming preference toward a singular time, but this does show the distribution of popular times to start playing throughout the day and can give insight towards which time frames have the most traffic.

In [14]:
hours_scatter = alt.Chart(sessions_data).mark_point().encode(
    x=alt.X('hour').title('Hour of the day'),
    y=alt.Y('count()').title('Number of players')
).properties(title='Distribution of player count throughout the day')
hours_scatter

alt.Chart(...)

This visualization shows a scatter plot version of the previous plot to visualize the actual distribution of points. I did not bin this plot because I wanted to see the distribution for all points throughout the entire day.

In [15]:
hours_line = alt.Chart(sessions_data).mark_line().encode(
    x=alt.X('hour').title('Hour of the day').bin(maxbins=20),
    y=alt.Y('count()').title('Number of players')
).properties(title='Distribution of player count throughout the day')
hours_line

alt.Chart(...)

This visualization shows a line plot of the previous two plots to better represent a trend rather than a comparison of amounts. This plot is binned so that I can see a general trend for the day.

In [16]:
sessions_data['start_date'] = sessions_data['start_time'].dt.date   
sessions_data['start_time_day'] = sessions_data['start_time'].dt.time
sessions_data['end_date'] = sessions_data['end_time'].dt.date 
sessions_data['end_time_day'] = sessions_data['end_time'].dt.time
sessions_data

,start_time,end_time,total_minutes_start,total_minutes_end,time_played_mins,hour,day,month,year,start_date,start_time_day,end_date,end_time_day
0,2024-06-30 18:12:00,2024-06-30 18:24:00,1092,1104.0,12.0,18,30,6,2024,2024-06-30,18:12:00,2024-06-30,18:24:00
1,2024-06-17 23:33:00,2024-06-17 23:46:00,1413,1426.0,13.0,23,17,6,2024,2024-06-17,23:33:00,2024-06-17,23:46:00
2,2024-07-25 17:34:00,2024-07-25 17:57:00,1054,1077.0,23.0,17,25,7,2024,2024-07-25,17:34:00,2024-07-25,17:57:00
3,2024-07-25 03:22:00,2024-07-25 03:58:00,202,238.0,36.0,3,25,7,2024,2024-07-25,03:22:00,2024-07-25,03:58:00
4,2024-05-25 16:01:00,2024-05-25 16:12:00,961,972.0,11.0,16,25,5,2024,2024-05-25,16:01:00,2024-05-25,16:12:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2024-05-10 23:01:00,2024-05-10 23:07:00,1381,1387.0,6.0,23,10,5,2024,2024-05-10,23:01:00,2024-05-10,23:07:00
1531,2024-07-01 04:08:00,2024-07-01 04:19:00,248,259.0,11.0,4,1,7,2024,2024-07-01,04:08:00,2024-07-01,04:19:00
1532,2024-07-28 15:36:00,2024-07-28 15:57:00,936,957.0,21.0,15,28,7,2024,2024-07-28,15:36:00,2024-07-28,15:57:00
1533,2024-07-25 06:15:00,2024-07-25 06:22:00,375,382.0,7.0,6,25,7,2024,2024-07-25,06:15:00,2024-07-25,06:22:00


In [17]:
tidy_sessions = sessions_data.drop(columns = ['start_time','end_time','start_date','end_date'])
tidy_sessions

,total_minutes_start,total_minutes_end,time_played_mins,hour,day,month,year,start_time_day,end_time_day
0,1092,1104.0,12.0,18,30,6,2024,18:12:00,18:24:00
1,1413,1426.0,13.0,23,17,6,2024,23:33:00,23:46:00
2,1054,1077.0,23.0,17,25,7,2024,17:34:00,17:57:00
3,202,238.0,36.0,3,25,7,2024,03:22:00,03:58:00
4,961,972.0,11.0,16,25,5,2024,16:01:00,16:12:00
...,...,...,...,...,...,...,...,...,...
1530,1381,1387.0,6.0,23,10,5,2024,23:01:00,23:07:00
1531,248,259.0,11.0,4,1,7,2024,04:08:00,04:19:00
1532,936,957.0,21.0,15,28,7,2024,15:36:00,15:57:00
1533,375,382.0,7.0,6,25,7,2024,06:15:00,06:22:00


## 4. Methods and Plan
- For answering a classification problem where an unknown observation (time of day) is given and one must predict the traffic/popularity of that time, we will use K Nearest Neighbors and regression to classify the unknown observation as being a time of high or low traffic. Regression using KNN was chosen because in this case, the classification will be numerical and the value will determine how many players will be playing at a given time slot. Based on the exploratory visualizations, the trend also does not appear linear so linear regression will likely be less effective in this case.
- In addition to this, we will also create visualizations (bar graphs, scatter, and line plots) to visually see the comparison between different sections of time. These visualizations will be important for clearly seeing the distribution of players throughout the day. Bar graphs will be used to compare amounts in specific quantities of time, the scatter plot will be used for the KNN regression analysis, and line plots will be used to show the general trend throughout the day.
- We are assuming that each data point is an independent session, not accounting for repeat players and therefore ignoring player habits or other dependencies on the player. We are assuming that there are no significant influences affecting player activity such as server downtime or game updates. We are also assuming that the player is actively playing during the session they are active rather than leaving the session idle or taking breaks during it (assuming that the time played is correlated to player engagement). We are also assuming there are no time zone varabilities in the dataset. 
- Limitations: the data is very large compared to datasets we have used in worksheets, which could be computationally costly and crossfitting the data will also take a long time. Using KNN regression might be limited by the density of the data points, a lack of interpretability, and potential influence of outliers. 
- For the regression model, we will do a 75% training data 25% test data split, then split the data into a 5 fold cross validation with validation sets. The training and test split will be done before any scaling to prevent data leakage. We will use RMSPE to test the effectiveness of our model. 